In [13]:
import numpy as np
import pandas as pd
from pgmpy.readwrite import XMLBIFReader
import random
import copy

In [14]:
# Settings
from discriminativeBN import DiscriminativeBN
from simplexMethod import NelderMeadOptimizer
from hookeJeeves import HookeJeevesOptimizer
xml_file = "models/chessTAN.xml" 
csv_file = "datasets/chess_data_3196.csv" 
target_var = "Class"        

# 1. Load BN Wrapper
# Note: Ensure your XML and CSV have matching column names/outcomes
bn = DiscriminativeBN(xml_file, target_var)

# 2. Load Data
# For demo purposes, create dummy data if file doesn't exist
try:
    df = pd.read_csv(csv_file)
except:
    print("CSV not found, creating dummy data for test...")
    # Create fake data matching the structure logic
    data_dict = {
        'x1': ['t', 'f', 't', 't'],
        'x2': ['f', 't', 'f', 'f'],
        'Class': ['won', 'nowin', 'won', 'won']
    }
    # Add other nodes if they exist in your XML
    for n in bn.nodes:
        if n not in data_dict:
            data_dict[n] = ['f'] * 4 
    df = pd.DataFrame(data_dict)

# 3. Define Objective Function
# We want to Maximize CLL, but Optimizer Minimizes. So minimize Negative CLL.
def objective(betas):
    cll = bn.calculate_cll(betas, df)
    return -cll # Negative because we minimize

# 4. Initialize Parameters (Betas)
# Start with random small values close to 0 (implies roughly uniform probs)
np.random.seed(42)
start_betas = np.random.normal(0, 0.1, bn.total_params)

print(f"Starting Optimization with {bn.total_params} parameters...")

# 5. Run Simplex
optimizer = HookeJeevesOptimizer(objective, dim=bn.total_params, max_iter=10) # Low iters for demo
best_betas = optimizer.optimize(start_betas)

# 6. Result
print("\nOptimization Complete.")
final_cll = bn.calculate_cll(best_betas, df)
print(f"Final Conditional Log Likelihood: {final_cll:.4f}")

# Show one resulting table
final_tables = bn.betas_to_probabilities(best_betas)
print(f"\nLearned Table for {target_var}:")
print(final_tables[target_var])

import pickle

with open("final_tablesChessTAN_hooke.pkl", "wb") as f:
    pickle.dump(final_tables, f)

Starting Optimization with 290 parameters...
--- Starting Hooke-Jeeves Optimization (Dim: 290) ---
Initial Score: -2253.3483 (CLL)
------------------------------
Iter 1: Best: -2253.3483 | Step: 0.05000 | Action: PATTERN MOVE (Accepted). New Score: -1419.9277
Iter 2: Best: -1419.9277 | Step: 0.05000 | Action: PATTERN MOVE (Accepted). New Score: -970.5924
Iter 3: Best: -970.5924 | Step: 0.05000 | Action: PATTERN MOVE (Accepted). New Score: -658.1991
Iter 4: Best: -658.1991 | Step: 0.05000 | Action: PATTERN MOVE (Accepted). New Score: -461.2604
Iter 5: Best: -461.2604 | Step: 0.05000 | Action: PATTERN MOVE (Accepted). New Score: -329.2005
Iter 6: Best: -329.2005 | Step: 0.05000 | Action: PATTERN MOVE (Accepted). New Score: -268.0554
Iter 7: Best: -268.0554 | Step: 0.05000 | Action: PATTERN MOVE (Accepted). New Score: -234.3870
Iter 8: Best: -234.3870 | Step: 0.05000 | Action: PATTERN MOVE (Accepted). New Score: -216.3843
Iter 9: Best: -216.3843 | Step: 0.05000 | Action: PATTERN MOVE (Acc

In [7]:
start_betas = np.random.normal(0, 0.1, bn.total_params)

In [16]:
best_betas
with open("final_betasChessTAN_hooke.pkl", "wb") as f:
    pickle.dump(best_betas, f)

In [17]:
final_tables

{'x1': array([[9.99858561e-01, 1.41439387e-04],
        [6.48503095e-01, 3.51496905e-01],
        [9.95178569e-03, 9.90048214e-01],
        [1.65367368e-01, 8.34632632e-01]]),
 'x2': array([[3.09768150e-01, 6.90231850e-01],
        [9.99499914e-01, 5.00085529e-04],
        [1.05790842e-01, 8.94209158e-01],
        [4.42406241e-03, 9.95575938e-01]]),
 'x3': array([[0.72457944, 0.27542056],
        [0.72120124, 0.27879876],
        [0.00716855, 0.99283145],
        [0.01097855, 0.98902145]]),
 'x4': array([[0.19536321, 0.80463679],
        [0.81717604, 0.18282396],
        [0.01235949, 0.98764051],
        [0.04541655, 0.95458345]]),
 'x5': array([[0.92445119, 0.07554881],
        [0.71184032, 0.28815968],
        [0.78865771, 0.21134229],
        [0.13557908, 0.86442092]]),
 'x6': array([[0.33330156, 0.66669844],
        [0.00681728, 0.99318272],
        [0.97402069, 0.02597931],
        [0.97202918, 0.02797082]]),
 'x7': array([[0.95733056, 0.04266944],
        [0.04612914, 0.95387086]

In [18]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix

def predict_row(bn, tables, row):
    """
    Predicts the Class for a single row of data.
    bn: The DiscriminativeBN object (needed for structure/parents)
    tables: The dictionary of learned probabilities (final_tables)
    row: A dictionary or pandas Series containing the features (x1, x2, ...)
    """
    target = bn.target_col
    target_outcomes = bn.var_info[target]['outcomes'] # e.g., ['f', 't'] or ['nowin', 'won']
    
    best_class = None
    best_log_prob = -float('inf')
    
    # Try every possible class value
    for t_val in target_outcomes:
        # Create a temporary row with this class assumption
        temp_row = row.copy()
        temp_row[target] = t_val
        
        # Calculate Log Joint Probability for this assumption
        # Log P(All Variables) = Sum( Log P(Node | Parents) )
        current_log_prob = 0
        
        for node in bn.nodes:
            # 1. Identify which row in the probability table to look at (based on parents)
            p_idx = bn._get_parent_config_index(temp_row, node)
            
            # 2. Identify which column (outcome) to look at
            # We must handle cases where test data has values not seen in training
            try:
                val_idx = bn.var_info[node]['outcomes'].index(str(temp_row[node]))
            except ValueError:
                # Fallback for unseen values (rare, but good for safety)
                val_idx = 0 
            
            # 3. Get probability
            prob = tables[node][p_idx, val_idx]
            current_log_prob += np.log(prob + 1e-10) # Avoid log(0)
            
        # Keep track of the best class
        if current_log_prob > best_log_prob:
            best_log_prob = current_log_prob
            best_class = t_val
            
    return best_class

def calculate_accuracy(bn, tables, df):
    """
    Iterates over the entire DataFrame, predicts, and returns accuracy.
    """
    y_true = df[bn.target_col].astype(str).tolist()
    y_pred = []
    
    # Iterate over every row
    records = df.to_dict('records')
    for row in records:
        prediction = predict_row(bn, tables, row)
        y_pred.append(prediction)
        
    # Calculate metrics
    acc = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred, labels=bn.var_info[bn.target_col]['outcomes'])
    
    return acc, cm, y_pred

# ==========================================
# 3. Usage Example
# ==========================================

# Assuming you ran the optimization from the previous step:
# bn = DiscriminativeBN(...)
# best_betas = ... from optimizer
# final_tables = bn.betas_to_probabilities(best_betas)
# df = pd.read_csv("chess_data.csv")

print("\n--- Starting Evaluation ---")

# Run prediction
accuracy, conf_matrix, predictions = calculate_accuracy(bn, final_tables, df)

print(f"Accuracy: {accuracy * 100:.2f}%")
print("\nConfusion Matrix:")
print(conf_matrix)

# Example of a single prediction comparison
print(f"\nRow 0 Actual: {df.iloc[0][bn.target_col]}")
print(f"Row 0 Pred  : {predictions[0]}")


--- Starting Evaluation ---
Accuracy: 98.06%

Confusion Matrix:
[[1639   30]
 [  32 1495]]

Row 0 Actual: won
Row 0 Pred  : won
